<a class="anchor" id="0.1"></a>
# **Prior Uncertainty Quantification (UQ) tools**


This notebook contains code examples to quantify uncertainties in precipitation data, including:
1. [variability across products](#variability)
1. [random errors in individual products](#randomerrors)
1. [consistency between data pairs](#consistency)

<div><img src="UQ.png" width="700"></div>

## import necessary libraries

In [1]:
import os 
import numpy as np
import pandas as pd
import glob  
import gdal
import shutil
import rasterio as rio 
import itertools     
from itertools import combinations 
from sklearn.metrics import r2_score, mean_squared_error
#import watertools modules
import watertools.General.raster_conversions as RC  
import watertools.General.data_conversions as DC  

/Users/rmourad/anaconda3/envs/env_wb/lib/python3.8/site-packages/paramiko/transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


# 1. variability across products <a class="anchor" id="variability"></a> 
[Uncertainty Quantification (UQ) tools](#0.1)

In [2]:
def extract_file_info(f_path):
    '''
    this function gets geospatial information using a raster as a template

    inputs:
    ----------
    - f_path : path to the location of the raster template (str)

     outputs:
    ----------
    - geo_out: geospatial info
    - proj: projection 
    - size_X: grid size in the X dimension
    - size_Y: grid size in the Y dimension
    
    '''
    in_fld = sorted(glob.glob(f_path + '/*.tif')) 
    #print(in_fld)
    template = in_fld[0] 
    print(template)
    
    geo_out, proj, size_X, size_Y= RC.Open_array_info(template) #using watertools pkg

    return geo_out, proj, size_X, size_Y


In [3]:
f_path=r'../data/p/chirps'
geo_out, proj, size_X, size_Y= extract_file_info(f_path)

../data/p/chirps/P_CHIRPS.v2.0_mm-month-1_monthly_2003.01.01.tif


In [4]:
def calc_annual_avg(root_folder, syear, eyear,output_f):
    """
    this function calculates the long-term annual average of tiff files 

    inputs:
    ----------
    - root_folder : path to the root folder (str)
    - syear : start year(int) 
    - eyear : end year (int)
    - output_f : location of the output folder (str)

    """
   
    for fx in os.listdir(root_folder):
        fx_path = os.path.join(root_folder, fx) 
        if os.path.isdir(fx_path):
            fx_sum_arr = None
            files = []
            num_yrs = eyear - syear + 1
            for rx in os.listdir(fx_path):
                rx_path = os.path.join(fx_path, rx)
                dest_arr = gdal.Open(rx_path)
                arr = dest_arr.GetRasterBand(1).ReadAsArray()

                if fx_sum_arr is None:
                    fx_sum_arr = np.zeros_like(arr)
                fx_sum_arr += arr

                files.append(rx_path)  

            annual_avg = fx_sum_arr / num_yrs #long-term annual avg array 
            out_raster = os.path.join(output_f, f"{fx}_longterm_avg.tif")
            DC.Save_as_tiff(out_raster, annual_avg,  geo_out, proj)


In [5]:
output_f=r'../output_folder/longterm_avg/'
root_folder=r'../data/p/'
calc_annual_avg(root_folder,2003,2019,output_f)

In [6]:
def calc_cv(input_fhs, output_f, label="CV_p_mm-2003-2019.tif"):
    """
    this calculates the coefficient of variation (%) across a list long-term annual tiff files

     inputs:
    ----------
    - input_fhs: List of file paths for input rasters  (list)
    - output_f : path to the output folder (str)
    - label: name of the output raster (str)
    """
    arrs = []

    for i in range(len(input_fhs)):
        path_in = input_fhs[i]
        print(path_in)
        dest_arrs = gdal.Open(path_in)
        arr = dest_arrs.GetRasterBand(1).ReadAsArray()
        arrs.append(arr)
        print(arr.shape)

    arrs = np.array(arrs) 
    avg= np.mean(arrs, axis=0) 
    var = np.mean((arrs - avg) ** 2, axis=0) 
    std = np.sqrt(var) #std
    cv = (std / avg) * 100 #CV(%)

    fh_out = os.path.join(output_f, label)
    DC.Save_as_tiff(fh_out, cv,  geo_out, proj)


In [7]:
input_fhs=sorted(glob.glob(output_f+'/*.tif'))
output_cv=r'../output_folder/cv'
calc_cv (input_fhs, output_cv, label="CV_p_mm-2003-2019.tif")

../output_folder/longterm_avg/chirps_longterm_avg.tif
(37, 13)
../output_folder/longterm_avg/gpm_longterm_avg.tif
(37, 13)
../output_folder/longterm_avg/mswep_longterm_avg.tif
(37, 13)
../output_folder/longterm_avg/trmm_longterm_avg.tif
(37, 13)


# 2. random errors in individual products <a class="anchor" id="randomerrors"></a> 
[Uncertainty Quantification (UQ) tools](#0.1)

In [8]:
def list_dirs(path_root):
    """
    inputs:
    ----------
    - path_root: path for dirs (str) 
    
    outputs:
    ----------
    - list of dirs paths

    """
    dirs_data = [os.path.join(path_root, dx) for dx in os.listdir(path_root)]
    return dirs_data

def make_dict_data(dirs_data):
    """
    inputs:
    ----------
    - dirs_data: list of data dirs paths
    
    outputs:
    ----------
    - dict_data: a dictionary with dir names as keys and file paths as values
    - dict_ts: a dictionary of stacke timeseries data 
    
    """
    dict_data = {}
    dict_ts = {}


    for dx in dirs_data:
        if not os.path.isdir(dx):
            continue
        fxs_list = [os.path.join(dx, f) for f in sorted(os.listdir(dx)) ] #list of files
        dir_label= os.path.split(dx)[1]
        dict_data[dir_label] = fxs_list
        
    #CHECK WHETHER ARRAYS HAVE CONSISTENT LENGTHS
    leng = {label: len(paths) for label, paths in dict_data.items()}  
    #print(leng) 

        
    for k in dict_data.keys():
        #print(k)
        for fx in range(len(fxs_list)):
            arr_3d = np.dstack([rio.open(d).read(1) for d in dict_data[k]])
            dict_ts[k] = arr_3d
    
    return dict_data,dict_ts



In [9]:
dirs_data=list_dirs("../data/p/")

In [10]:
dict_data,dict_ts=make_dict_data(dirs_data)

In [11]:
def linear_rescaling(source_dataset, ref_dataset):
    '''
    this function scales the source data set to the mean and std of a reference data set

    inputs:
    ----------
    - source_dataset to be scaled 
    - ref_dataset to be used as template

    outputs:
    ----------
    - scaled dataset
    
    references
    ----------
    linear rescaling Eq 2 in:
    --Brocca, L., Melone, F., Moramarco, T., Wagner, W., & Hasenauer, S. (2010). 
      ASCAT soil wetness index validation through in situ and 
      modeled soil moisture data in central Italy. 
      Remote Sensing of Environment, 114(11), 2745-2755.
      
    other alternative rescaling options include: 
    - forcing source data set to have the same min and max as the ref data set
    - matching the cumulative distribution function (CDF) of the source data set with that of ref data set
    as in Eqs 6 & 7 in: 
    --Wu, K., Ryu, D., Wagner, W., & Hu, Z. (2023).
      A global-scale intercomparison of Triple Collocation Analysis-and ground-based 
      soil moisture time-variant errors derived from different rescaling techniques.
      Remote Sensing of Environment, 285, 113387.
    
    '''
    return ((source_dataset - np.nanmean(source_dataset)) / np.nanstd(source_dataset)) * np.nanstd(ref_dataset) + np.nanmean(ref_dataset)


def compute_random_errors(A,B,C):
    '''
    this function computes random errors as std of the noise error in individual products 
    
    inputs:
    ----------
    - triplet data sets: A, B & C 
     where A, B and C are three spatially and temporally collocated data sets
    
    outputs:
    ----------
    - error std
    
    '''
    
    std_A = np.sqrt(np.abs(np.nanmean((A - B) * (A - C))))
    std_B = np.sqrt(np.abs(np.nanmean((B - A) * (B - C))))
    std_C = np.sqrt(np.abs(np.nanmean((C - A) * (C - B))))

    return std_A, std_B, std_C

def compute_errs_combos(dict_ts, combos):
    '''
    this function computes errors for all combinations of triplets 
    a wrapper for compute_random_errors
    
    inputs:
    ----------
    - dict_ts:timeseries data (dict)
    - combos: all possible combinations of triplets
    
    outputs:
    ----------
    - errors: spatial std of errors in individual product 
    
    references
    ----------
    – To solve for the standard deciation, std is computed as proposed by Stoffelen (1998)** 
    – by averaging the cross-multiplied differences between the three a-priori rescaled data sets.
    - Stoffelen (1998) also proposed other formulation based on combinations of the covariances
      between the data sets. However, both approaches are mathematically identical.
    
    **Stoffelen, A. (1998). Toward the true near‐surface wind speed: Error modeling and calibration 
    using triple collocation.Journal of geophysical research: oceans, 103(C4), 7755-7766.
    
    '''
    errs = {f'{map_combo[0]}_{z}': {
                f'{map_combo[0]}_{z}': np.zeros_like(dict_ts[map_combo[0]][:, :, 0]),
                f'{map_combo[1]}_{z}': np.zeros_like(dict_ts[map_combo[1]][:, :, 0]),
                f'{map_combo[2]}_{z}': np.zeros_like(dict_ts[map_combo[2]][:, :, 0])} 
              for z, map_combo in enumerate(combos)}

    for i in range(errs[f'{combos_labels[0]}_0'][f'{combos_labels[0]}_0'].shape[0]):
        for j in range(errs[f'{combos_labels[0]}_0'][f'{combos_labels[0]}_0'].shape[1]):
            for x, map_combo in enumerate(combos):
                A_label = f'{map_combo[0]}_{x}'
                val_A = dict_ts[map_combo[0]][i, j, :]
                val_B = dict_ts[map_combo[1]][i, j, :]
                val_C = dict_ts[map_combo[2]][i, j, :]
                
                std_A, std_B, std_C= compute_random_errors(val_A, linear_rescaling(val_B, val_A), linear_rescaling(val_C, val_A))
                
                errs[A_label][f'{map_combo[0]}_{x}'][i, j] = std_A
                errs[A_label][f'{map_combo[1]}_{x}'][i, j] = std_B
                errs[A_label][f'{map_combo[2]}_{x}'][i, j] = std_C

    return errs


In [12]:
combos_labels = ['chirps', 'gpm', 'mswep','trmm']  
map_combo = list(itertools.permutations(combos_labels, 3))

random_errors = compute_errs_combos(dict_ts, map_combo) #calc errors for all combos


/var/folders/k7/s9nsfw2n2xn2wt1s8hdszh8m0000gn/T/ipykernel_41284/879367388.py:32: RuntimeWarning: Mean of empty slice
  return ((source_dataset - np.nanmean(source_dataset)) / np.nanstd(source_dataset)) * np.nanstd(ref_dataset) + np.nanmean(ref_dataset)
/Users/rmourad/anaconda3/envs/env_wb/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1872: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/var/folders/k7/s9nsfw2n2xn2wt1s8hdszh8m0000gn/T/ipykernel_41284/879367388.py:50: RuntimeWarning: Mean of empty slice
  std_A = np.sqrt(np.abs(np.nanmean((A - B) * (A - C))))
/var/folders/k7/s9nsfw2n2xn2wt1s8hdszh8m0000gn/T/ipykernel_41284/879367388.py:51: RuntimeWarning: Mean of empty slice
  std_B = np.sqrt(np.abs(np.nanmean((B - A) * (B - C))))
/var/folders/k7/s9nsfw2n2xn2wt1s8hdszh8m0000gn/T/ipykernel_41284/879367388.py:52: RuntimeWarning: Mean of empty slice
  std_C = np.sqrt(np.abs(np.nanmean((C - A) * (C - B))))


In [13]:
def save_errors_to_tifs(out_f, random_errors):
    '''
    this function outputs errors to tiff files

    inputs:
    ----------
    - out_f : output folder (str)
    - random_errors (dict) 

    '''
    for k,val in random_errors.items():
        for arr_k, arr_data in val.items():
            label_outfile = os.path.join(out_f, f"{arr_k}.tif")
            DC.Save_as_tiff(label_outfile, arr_data, geo_out, proj)

In [14]:
output_folder=r'../output_folder/TC'
save_errors_to_tifs(output_folder,random_errors)

In [15]:
def store_rasters_in_subfold(f_path, ext='.tif', extr=2):
    '''
    this function stores tiff files in subfolders using their labels

    inputs:
    ----------
    - f_path : path to tiff files
    - ext : tiff files extension (str)
    - extr : characters extracted from the filelabel(int) 

    '''
    for filelabel in os.listdir(f_path):
        if filelabel.endswith(ext):
            label= filelabel[:extr]
            subfold = os.path.join(f_path, label)
            os.makedirs(subfold, exist_ok=True)
            shutil.move(os.path.join(f_path, filelabel), os.path.join(subfold, filelabel))
            

In [16]:
store_rasters_in_subfold(output_folder)

In [17]:
#rename the subfolders

for f_label in os.listdir(output_folder):
    if f_label == 'ch':
        os.rename(os.path.join(output_folder, f_label), os.path.join(output_folder, 'chirps'))
    elif f_label == 'ms':
        os.rename(os.path.join(output_folder, f_label), os.path.join(output_folder, 'mswep'))
    elif f_label == 'tr':
        os.rename(os.path.join(output_folder, f_label), os.path.join(output_folder, 'trmm'))
    elif f_label == 'gp':
        os.rename(os.path.join(output_folder, f_label), os.path.join(output_folder,'gpm'))


In [18]:
def compute_avg_error(root_folder):
    '''
    this function computes the average error for list of triple collocation error tiff files

    inputs:
    ----------
    root_folder: path to the error tiff files (str)
    
    '''
    for fhs_label in os.listdir(root_folder):
        fhs = os.path.join(root_folder, fhs_label)

        if os.path.isdir(fhs):
            sum_arr = None
            fr = []

            for rx in os.listdir(fhs):
                rx_path = os.path.join(fhs, rx)
                #print(rx_path)
                fr.append(rx_path)  # Append the raster file path to the list
                img = gdal.Open(rx_path)  # Open the raster file
                arr = img.GetRasterBand(1).ReadAsArray()

                # update sum_arr
                if sum_arr is None:
                    sum_arr = np.zeros_like(arr)

                sum_arr += arr

            # calculate the avg
            arr_avg = sum_arr / len(fr)

            geo_out, proj, size_X, size_Y = extract_file_info(f_path)

            # save output
            output_label = os.path.join(fhs, f"TC_{fhs_label}_avg.tif")
            DC.Save_as_tiff(output_label, arr_avg, geo_out, proj)  # Save raster 


In [19]:
compute_avg_error(output_folder)

../data/p/chirps/P_CHIRPS.v2.0_mm-month-1_monthly_2003.01.01.tif
../data/p/chirps/P_CHIRPS.v2.0_mm-month-1_monthly_2003.01.01.tif
../data/p/chirps/P_CHIRPS.v2.0_mm-month-1_monthly_2003.01.01.tif
../data/p/chirps/P_CHIRPS.v2.0_mm-month-1_monthly_2003.01.01.tif


# 3. consistency between data pairs <a class="anchor" id="consistency"></a> 
[Uncertainty Quantification (UQ) tools](#0.1)

In [20]:
def compute_monthly_avg(r_f, outf_p, syear, eyear):
    """
    this function computes the long-term monthly average of tiff files

    inputs:
    ----------
    - r_f: path to the root folder (str)
    - outf_p : output folder to store avg monthly tiff files(str)
    - syear : start year(int)
    - eyear : end year(int)
    """
    yrs_nb = eyear - syear + 1
    mths = range(1, 13)

    for subf_label in os.listdir(r_f):
        fhs = os.path.join(r_f, subf_label)

        if os.path.isdir(fhs):
            m_avgs = {}

            for rx in os.listdir(fhs):
                rx_path = os.path.join(fhs, rx)

                dest_arr = gdal.Open(rx_path)
                arr = dest_arr.GetRasterBand(1).ReadAsArray()

                m = int(rx[-9:-7])
                #print(m)

                # aggregate
                if m in m_avgs:
                    m_avgs[m].append(arr)
                else:
                    m_avgs[m] = [arr]
                    
            fhs_output= os.path.join(outf_p, subf_label)
            os.makedirs(fhs_output, exist_ok=True)

            for m,x in m_avgs.items():
                m_mean = np.mean(x, axis=0)
                output_label = os.path.join(fhs_output, f'{subf_label}_avg_{m:02d}.tif')
                DC.Save_as_tiff(output_label, m_mean, geo_out, proj)


In [21]:
outf_p=r'../output_folder/monthly_longterm_avg'
compute_monthly_avg(root_folder, outf_p, 2003, 2019)

In [22]:
#extract pixel by pixel value from long-term annual rasters
# df
output_f=r'../output_folder/longterm_avg/'
output_f_p=sorted(glob.glob(output_f+'/*.tif')) #get list of tif files in the output_f folder
print(output_f_p)
df = pd.DataFrame(columns=['row', 'col'] + [os.path.splitext(os.path.basename(rx))[0] + '_val' for rx in output_f_p], dtype='object')

for i, rx in enumerate(output_f_p):
    #print(rx)
    # Open the raster file
    img = gdal.Open(rx)
    data = img.GetRasterBand(1).ReadAsArray()  
    idx, jdx = data.shape
    #print(idx, jdx)
    i_ind, j_ind = zip(*[(i, j) for i in range(idx) for j in range(jdx)])
    
    rx_label = os.path.splitext(os.path.basename(rx))[0]
    df[f'{rx_label}_val'] = data.flatten()
    df['row'] = i_ind
    df['col'] = j_ind 

df_annual = df[['row', 'col'] + [f'{os.path.splitext(os.path.basename(rx))[0]}_val' for rx in output_f_p]]
df_annual=df_annual.dropna()
print(df_annual)

['../output_folder/longterm_avg/chirps_longterm_avg.tif', '../output_folder/longterm_avg/gpm_longterm_avg.tif', '../output_folder/longterm_avg/mswep_longterm_avg.tif', '../output_folder/longterm_avg/trmm_longterm_avg.tif']
     row  col  chirps_longterm_avg_val  gpm_longterm_avg_val  \
11     0   11              1419.768066           1466.187256   
23     1   10              1418.300293           1406.040649   
24     1   11              1412.772339           1427.168213   
35     2    9              1263.947266           1352.552002   
36     2   10              1311.729614           1375.716309   
..   ...  ...                      ...                   ...   
446   34    4               861.020447            884.861084   
447   34    5               852.606934            885.286255   
459   35    4               836.936768            868.385376   
460   35    5               828.406067            870.664551   
472   36    4               844.303040            854.458862   

     msw

In [23]:
#extract pixel by pixel value from long-term monthly rasters
outfolder_P = r'../output_folder/monthly_longterm_avg'
list_data = []
# Loop over subfolders in the main folder
for fhs in os.listdir(outfolder_P):
    fhs_path = os.path.join(outfolder_P, fhs)
    rx_data = sorted(glob.glob(os.path.join(fhs_path, '*.tif')))

    for i, rx in enumerate(rx_data):
        img = gdal.Open(rx)
        data= img.GetRasterBand(1).ReadAsArray()
        idx, jdx = data.shape
        i_ind, j_ind = zip(*[(i, j) for i in range(idx) for j in range(jdx)])
        m = os.path.splitext(os.path.basename(rx))[0][-2:] #extract month
        list_data.extend(list(zip(i_ind, j_ind, [m] * len(i_ind), [f'{fhs}_val'] * len(i_ind), data.flatten())))

# df
col_labels = ['row', 'col', 'month', 'var', 'val']
df_month = pd.DataFrame(list_data, columns=col_labels)
df_month = df_month.pivot_table(index=['row', 'col', 'month'], columns='var', values='val').reset_index()
print(df_month.dropna())


var   row  col month  chirps_val     gpm_val   mswep_val    trmm_val
0       0   11    01   37.980198   59.296261   41.447906   62.900562
1       0   11    02   46.546814   46.641949   49.457245   61.216122
2       0   11    03   44.627460   43.122433   38.775211   40.940525
3       0   11    04   21.511440   30.202282   26.348068   32.613499
4       0   11    05   30.734367   39.504498   36.736843   36.886265
...   ...  ...   ...         ...         ...         ...         ...
2647   36    4    08  240.456345  231.957367  218.106476  180.060471
2648   36    4    09  180.250000  155.690353  127.589211  123.292236
2649   36    4    10   20.838272   18.132839   15.366764   17.345369
2650   36    4    11    3.061043    5.250890    4.227652    3.091552
2651   36    4    12    7.615211    3.761159    4.948952    3.583566

[2652 rows x 7 columns]


In [24]:
def compute_spatial_R2(A, B):
    A = np.array(A)
    B = np.array(B)
    a, b = np.polyfit(A, B, 1)
    B_pred = a * A + b
    R2 = r2_score(B, B_pred)
    return {'R2': R2}

def do_R2_annual_pairs(df):
    
    dict_yr = df.astype(float).to_dict('list') 
    combos = list(itertools.combinations(dict_yr, 2))

    res = {}

    for idx, i in enumerate(combos):
        A = dict_yr[i[0]]
        B = dict_yr[i[1]]
        R2_metric = compute_spatial_R2(A, B)
        res[i] = R2_metric
    return res

def do_R2_month_pairs(df):
    '''
    this function computes R2 metric by a month column applied to df
    '''
    res = {}

    # Group the data by month column
    m_grp = df.groupby('month')
    for m, grp in m_grp:
        dict_m = grp.astype(float).to_dict('list')
        combos = list(itertools.combinations(dict_m, 2))
        for idx, i in enumerate(combos):
            A = dict_m[i[0]]
            B = dict_m[i[1]]
            R2_metric = compute_spatial_R2(A, B)
            # store R2 with the month as an identifier
            res[(m, i)] = R2_metric
    return res


In [25]:
p_yr_dict = {'chirps_longterm_avg_val':'chirps',
             'gpm_longterm_avg_val': 'gpm',
             'mswep_longterm_avg_val':'mswep', 
             'trmm_longterm_avg_val':'trmm'}
    
p_yr_df= df_annual.loc[:, df_annual.columns.isin(p_yr_dict.keys())]
p_yr_df.rename(columns = p_yr_dict, inplace = True)
p_yr_df

/var/folders/k7/s9nsfw2n2xn2wt1s8hdszh8m0000gn/T/ipykernel_41284/191048470.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  p_yr_df.rename(columns = p_yr_dict, inplace = True)


,chirps,gpm,mswep,trmm
11,1419.768066,1466.187256,1691.985718,1466.706055
23,1418.300293,1406.040649,1555.549561,1399.857178
24,1412.772339,1427.168213,1593.652588,1424.922363
35,1263.947266,1352.552002,1436.382935,1325.330200
36,1311.729614,1375.716309,1451.153320,1352.153320
...,...,...,...,...
446,861.020447,884.861084,839.233459,729.890198
447,852.606934,885.286255,850.931396,735.751831
459,836.936768,868.385376,824.186279,727.025879
460,828.406067,870.664551,835.187927,732.518066


In [26]:
p_yr_df

,chirps,gpm,mswep,trmm
11,1419.768066,1466.187256,1691.985718,1466.706055
23,1418.300293,1406.040649,1555.549561,1399.857178
24,1412.772339,1427.168213,1593.652588,1424.922363
35,1263.947266,1352.552002,1436.382935,1325.330200
36,1311.729614,1375.716309,1451.153320,1352.153320
...,...,...,...,...
446,861.020447,884.861084,839.233459,729.890198
447,852.606934,885.286255,850.931396,735.751831
459,836.936768,868.385376,824.186279,727.025879
460,828.406067,870.664551,835.187927,732.518066


In [27]:
do_R2_annual_pairs(p_yr_df)

{('chirps', 'gpm'): {'R2': 0.8135625653043942},
 ('chirps', 'mswep'): {'R2': 0.860958638868887},
 ('chirps', 'trmm'): {'R2': 0.8348133706694025},
 ('gpm', 'mswep'): {'R2': 0.9324396555332763},
 ('gpm', 'trmm'): {'R2': 0.9586354525927328},
 ('mswep', 'trmm'): {'R2': 0.9807892099038125}}

In [28]:
p_m_dict = {'chirps_val':'chirps',
             'gpm_val': 'gpm',
             'mswep_val':'mswep', 
             'trmm_val':'trmm',
             'month':'month'}
    
p_m_df= df_month.loc[:, df_month.columns.isin(p_m_dict.keys())]
p_m_df.rename(columns = p_m_dict, inplace = True)
p_m_df.set_index('month', inplace=True)
p_m_df

/var/folders/k7/s9nsfw2n2xn2wt1s8hdszh8m0000gn/T/ipykernel_41284/2019154241.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  p_m_df.rename(columns = p_m_dict, inplace = True)


var,chirps,gpm,mswep,trmm
month,,,,
01,37.980198,59.296261,41.447906,62.900562
02,46.546814,46.641949,49.457245,61.216122
03,44.627460,43.122433,38.775211,40.940525
04,21.511440,30.202282,26.348068,32.613499
05,30.734367,39.504498,36.736843,36.886265
...,...,...,...,...
08,240.456345,231.957367,218.106476,180.060471
09,180.250000,155.690353,127.589211,123.292236
10,20.838272,18.132839,15.366764,17.345369


In [29]:
do_R2_month_pairs(p_m_df)

{('01', ('chirps', 'gpm')): {'R2': 0.9117175176420835},
 ('01', ('chirps', 'mswep')): {'R2': 0.8380546688725639},
 ('01', ('chirps', 'trmm')): {'R2': 0.7100072656857284},
 ('01', ('gpm', 'mswep')): {'R2': 0.9577343192988783},
 ('01', ('gpm', 'trmm')): {'R2': 0.8862834214570792},
 ('01', ('mswep', 'trmm')): {'R2': 0.9653490511347538},
 ('02', ('chirps', 'gpm')): {'R2': 0.7156481131995412},
 ('02', ('chirps', 'mswep')): {'R2': 0.7182622937670498},
 ('02', ('chirps', 'trmm')): {'R2': 0.812094399837707},
 ('02', ('gpm', 'mswep')): {'R2': 0.9227742637918978},
 ('02', ('gpm', 'trmm')): {'R2': 0.9412955162195374},
 ('02', ('mswep', 'trmm')): {'R2': 0.9321551332270871},
 ('03', ('chirps', 'gpm')): {'R2': 0.7537432137583072},
 ('03', ('chirps', 'mswep')): {'R2': 0.8455371163069515},
 ('03', ('chirps', 'trmm')): {'R2': 0.7007273430903987},
 ('03', ('gpm', 'mswep')): {'R2': 0.9120727014302361},
 ('03', ('gpm', 'trmm')): {'R2': 0.8380646692561511},
 ('03', ('mswep', 'trmm')): {'R2': 0.736662498933





## Thanks for viewing!


